In [21]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score,accuracy_score,confusion_matrix, f1_score, precision_score, auc,roc_auc_score,roc_curve, precision_recall_curve
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import ClusterCentroids,NearMiss, RandomUnderSampler
from imblearn.combine import SMOTEENN,SMOTETomek
#from imblearn.ensemble import BalanceCascade
from sklearn.linear_model import LogisticRegression
#from sklearn.cross_validation import KFold, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix,precision_recall_curve,auc,roc_auc_score,roc_curve,recall_score,classification_report 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [22]:
data = pd.read_csv('data_source/log2.csv')
data.head()

,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Action,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received
0,57222,53,54587,53,allow,177,94,83,2,30,1,1
1,56258,3389,56258,3389,allow,4768,1600,3168,19,17,10,9
2,6881,50321,43265,50321,allow,238,118,120,2,1199,1,1
3,50553,3389,50553,3389,allow,3327,1438,1889,15,17,8,7
4,50002,443,45848,443,allow,25358,6778,18580,31,16,13,18


In [23]:
labelencoder=LabelEncoder()
for col in ['Source Port','Destination Port','NAT Source Port','NAT Destination Port']:
    data[col] = labelencoder.fit_transform(data[col])
 
data.tail(10)

,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Action,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received
65522,12195,3212,0,0,deny,66,66,0,1,0,1,0
65523,13790,1215,0,0,deny,66,66,0,1,0,1,0
65524,12688,1804,0,0,deny,62,62,0,1,0,1,0
65525,22516,10,14203,9,allow,356,118,238,2,30,1,1
65526,10740,2161,28976,1669,allow,70,70,0,2,8,2,0
65527,21002,15,5390,11,allow,314,192,122,6,15,4,2
65528,10093,15,5499,11,allow,4680740,67312,4613428,4675,77,985,3690
65529,13542,28,0,0,drop,70,70,0,1,0,1,0
65530,13541,28,0,0,drop,70,70,0,1,0,1,0
65531,13538,28,0,0,drop,70,70,0,1,0,1,0


In [24]:
data['Action'].replace({'allow':0, 'deny':1, 'drop':2, 'reset-both':3},inplace=True)

data.tail(10)

,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Action,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received
65522,12195,3212,0,0,1,66,66,0,1,0,1,0
65523,13790,1215,0,0,1,66,66,0,1,0,1,0
65524,12688,1804,0,0,1,62,62,0,1,0,1,0
65525,22516,10,14203,9,0,356,118,238,2,30,1,1
65526,10740,2161,28976,1669,0,70,70,0,2,8,2,0
65527,21002,15,5390,11,0,314,192,122,6,15,4,2
65528,10093,15,5499,11,0,4680740,67312,4613428,4675,77,985,3690
65529,13542,28,0,0,2,70,70,0,1,0,1,0
65530,13541,28,0,0,2,70,70,0,1,0,1,0
65531,13538,28,0,0,2,70,70,0,1,0,1,0


Check magnitude of imbalance

In [25]:
data['Action'].value_counts()

0    37640
1    14987
2    12851
3       54
Name: Action, dtype: int64

Split into training and test datasets

In [26]:
# split the data into train and test set
train, test = train_test_split(data, test_size=0.30, random_state=0,stratify=data['Action'])

In [27]:
X = data.iloc[:, data.columns != 'Action']
y = data.iloc[:, data.columns == 'Action']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, random_state = 0,stratify=data['Action'])

print(X_train.shape)
print(X_test.shape)

(45872, 11)
(19660, 11)


1. Imbalance data on train dataset

In [28]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(sampling_strategy='auto')
X_smt, y_smt = smt.fit_resample(X_train, y_train)

print(y.value_counts())
print(y_smt.value_counts())

Action
0         37640
1         14987
2         12851
3            54
dtype: int64
Action
0         26345
2         26343
3         26323
1         26319
dtype: int64


In [29]:
from imblearn.combine import SMOTEENN

sme = SMOTEENN(sampling_strategy='auto')
X_sme, y_sme = sme.fit_resample(X_train, y_train)

print(y.value_counts())
print(y_sme.value_counts())

Action
0         37640
1         14987
2         12851
3            54
dtype: int64
Action
2         26253
0         25836
3         25800
1         25266
dtype: int64


Train & evaluate

In [30]:
knn = KNeighborsClassifier(n_neighbors=4,algorithm='auto',weights='distance')
knn.fit(X_smt,y_smt)

train_yhat = knn.predict(X_smt)
train_f1 = f1_score(y_smt,train_yhat,average='weighted')

test_yhat = knn.predict(X_test)
test_f1 = f1_score(y_test,test_yhat,average='weighted')
print('Train Score\n',classification_report(y_smt,train_yhat))
print('Test Score\n',classification_report(y_test, test_yhat))
print(f"AVG F1-Score Train: {train_f1}\nAVG F1-Score Test: {test_f1}")

d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


Train Score
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     26345
           1       1.00      1.00      1.00     26319
           2       1.00      1.00      1.00     26343
           3       1.00      1.00      1.00     26323

    accuracy                           1.00    105330
   macro avg       1.00      1.00      1.00    105330
weighted avg       1.00      1.00      1.00    105330

Test Score
               precision    recall  f1-score   support

           0       1.00      0.99      0.99     11292
           1       0.99      0.96      0.97      4496
           2       0.98      1.00      0.99      3856
           3       0.04      0.50      0.07        16

    accuracy                           0.98     19660
   macro avg       0.75      0.86      0.76     19660
weighted avg       0.99      0.98      0.99     19660

AVG F1-Score Train: 0.9994113053408112
AVG F1-Score Test: 0.9874119553915355


In [31]:
knn = KNeighborsClassifier(n_neighbors=4,algorithm='auto',weights='distance')
knn.fit(X_sme,y_sme)

train_yhat = knn.predict(X_sme)
train_f1 = f1_score(y_sme,train_yhat,average='weighted')

test_yhat = knn.predict(X_test)
test_f1 = f1_score(y_test,test_yhat,average='weighted')
print('Train Score\n',classification_report(y_sme,train_yhat))
print('Test Score\n',classification_report(y_test, test_yhat))
print(f"AVG F1-Score Train: {train_f1}\nAVG F1-Score Test: {test_f1}")

d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


Train Score
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     25836
           1       1.00      1.00      1.00     25266
           2       1.00      1.00      1.00     26253
           3       1.00      1.00      1.00     25800

    accuracy                           1.00    103155
   macro avg       1.00      1.00      1.00    103155
weighted avg       1.00      1.00      1.00    103155

Test Score
               precision    recall  f1-score   support

           0       1.00      0.98      0.99     11292
           1       0.98      0.93      0.95      4496
           2       0.97      1.00      0.99      3856
           3       0.03      0.62      0.05        16

    accuracy                           0.97     19660
   macro avg       0.75      0.88      0.75     19660
weighted avg       0.99      0.97      0.98     19660

AVG F1-Score Train: 1.0
AVG F1-Score Test: 0.9807769608559571


2. Imbalance data on train&test dataset

In [32]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(sampling_strategy='auto')
X_smt_train, y_smt_train = smt.fit_resample(X_train, y_train)
X_smt_test, y_smt_test = smt.fit_resample(X_test, y_test)

print(y.value_counts())
print(y_smt_train.value_counts())
print(y_smt_test.value_counts())

Action
0         37640
1         14987
2         12851
3            54
dtype: int64
Action
2         26347
0         26342
3         26325
1         26322
dtype: int64
Action
2         11290
3         11283
0         11282
1         11275
dtype: int64


In [33]:
from imblearn.combine import SMOTEENN

sme = SMOTEENN(sampling_strategy='auto')
X_sme_train, y_sme_train = sme.fit_resample(X_train, y_train)
X_sme_test, y_sme_test = sme.fit_resample(X_test, y_test)

print(y.value_counts())
print(y_smt_train.value_counts())
print(y_smt_test.value_counts())

Action
0         37640
1         14987
2         12851
3            54
dtype: int64
Action
2         26347
0         26342
3         26325
1         26322
dtype: int64
Action
2         11290
3         11283
0         11282
1         11275
dtype: int64


Train & evaluate

In [34]:
knn = KNeighborsClassifier(n_neighbors=4,algorithm='auto',weights='distance')
knn.fit(X_smt_train,y_smt_train)

train_yhat = knn.predict(X_smt_train)
train_f1 = f1_score(y_smt_train,train_yhat,average='weighted')

test_yhat = knn.predict(X_smt_test)
test_f1 = f1_score(y_smt_test,test_yhat,average='weighted')
print('Train Score\n',classification_report(y_smt_train,train_yhat))
print('Test Score\n',classification_report(y_smt_test, test_yhat))
print(f"AVG F1-Score Train: {train_f1}\nAVG F1-Score Test: {test_f1}")

d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


Train Score
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     26342
           1       1.00      1.00      1.00     26322
           2       1.00      1.00      1.00     26347
           3       1.00      1.00      1.00     26325

    accuracy                           1.00    105336
   macro avg       1.00      1.00      1.00    105336
weighted avg       1.00      1.00      1.00    105336

Test Score
               precision    recall  f1-score   support

           0       0.92      0.99      0.95     11282
           1       0.72      0.95      0.82     11275
           2       0.99      1.00      0.99     11290
           3       0.93      0.55      0.69     11283

    accuracy                           0.87     45130
   macro avg       0.89      0.87      0.87     45130
weighted avg       0.89      0.87      0.87     45130

AVG F1-Score Train: 0.9993733727507159
AVG F1-Score Test: 0.8652569390841675


In [35]:
knn = KNeighborsClassifier(n_neighbors=4,algorithm='auto',weights='distance')
knn.fit(X_sme_train,y_sme_train)

train_yhat = knn.predict(X_sme_train)
train_f1 = f1_score(y_sme_train,train_yhat,average='weighted')

test_yhat = knn.predict(X_sme_test)
test_f1 = f1_score(y_sme_test,test_yhat,average='weighted')
print('Train Score\n',classification_report(y_sme_train,train_yhat))
print('Test Score\n',classification_report(y_sme_test, test_yhat))
print(f"AVG F1-Score Train: {train_f1}\nAVG F1-Score Test: {test_f1}")

d:\Users\User\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


Train Score
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     25841
           1       1.00      1.00      1.00     25221
           2       1.00      1.00      1.00     26256
           3       1.00      1.00      1.00     25809

    accuracy                           1.00    103127
   macro avg       1.00      1.00      1.00    103127
weighted avg       1.00      1.00      1.00    103127

Test Score
               precision    recall  f1-score   support

           0       0.93      1.00      0.96     10885
           1       0.72      0.95      0.82     10818
           2       1.00      1.00      1.00     11260
           3       0.92      0.57      0.70     11123

    accuracy                           0.88     44086
   macro avg       0.89      0.88      0.87     44086
weighted avg       0.89      0.88      0.87     44086

AVG F1-Score Train: 1.0
AVG F1-Score Test: 0.8707066123216891
